 - This Notebook is Created by **Mohd Muttalib**

- **Email** - muttalib1326@gmail.com



- **Contact** - +918445818187

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score,classification_report,confusion_matrix,log_loss

# Loading data

In [9]:
# loading the train data
df = pd.read_csv('training_set.csv',index_col=0)
df.head()

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X49,X50,X51,X52,X53,X54,X55,X56,X57,Y
0,0.00,0.00,4.34,0.00,0.00,0.00,0.00,0.00,0.0,0.00,...,0.0,0.000,0.0,1.342,0.000,0.000,1.200,2,12,0
1,0.00,0.56,0.56,0.00,1.12,0.56,2.25,0.00,0.0,0.56,...,0.0,0.083,0.0,0.503,0.000,0.083,16.304,148,375,1
2,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,...,0.0,0.000,0.0,0.000,0.000,0.000,1.000,1,5,0
3,0.64,0.00,0.64,0.00,1.93,0.00,0.00,0.00,0.0,0.00,...,0.0,0.000,0.0,0.462,0.370,0.000,2.440,22,122,1
4,0.58,0.00,0.00,35.46,0.58,0.00,0.58,0.58,0.0,0.00,...,0.0,0.000,0.0,0.239,0.239,0.000,3.338,123,207,1


In [10]:
# Loading Test data
test_data = pd.read_csv('test_set.csv',index_col=0)
test_data.head()

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X48,X49,X50,X51,X52,X53,X54,X55,X56,X57
0,0.70,0.0,0.70,0.0,0.00,0.00,0.00,0.00,0.00,0.00,...,0.0,0.0,0.000,0.0,0.105,0.000,0.000,2.342,47,89
1,0.00,0.0,0.84,0.0,0.84,0.00,0.84,0.00,0.00,0.00,...,0.0,0.0,0.388,0.0,0.776,0.129,0.000,10.375,168,249
2,0.46,0.3,0.46,0.0,0.05,0.12,0.05,0.28,0.43,0.74,...,0.0,0.0,0.065,0.0,0.325,0.756,0.153,5.891,193,3040
3,0.10,0.2,1.01,0.0,0.80,0.80,0.50,0.00,0.80,0.10,...,0.0,0.0,0.110,0.0,0.490,0.158,0.015,8.550,669,1351
4,0.00,0.0,0.72,0.0,0.72,0.00,0.72,0.00,0.00,0.00,...,0.0,0.0,0.364,0.0,0.729,0.121,0.000,7.781,32,249


# Splitting into X and y

In [11]:
X = df.drop(['Y'],axis=1)
y = df['Y']

# Train Test Split

In [12]:
X_train, X_test, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Using RandomForest Classifier for feature Selection

In [13]:
clf = RandomForestClassifier(100, max_depth=None, n_jobs=-1)
clf.fit(X_train,y_train)
feature_importance = clf.feature_importances_

In [14]:
# Ranking the features with their respect to feature importances
fi = sorted(zip(X.columns,feature_importance),key=lambda x: x[1], reverse=True)
# Extracting Top 30 features
top_features = [x[0] for x in fi[:30]]

In [15]:
# Selecting the top features from data
X_train_dash = X_train[top_features]
X_test_dash = X_test[top_features]

In [16]:
# Selecting the top features for test data
test_data_dash = test_data[top_features]

# Normalizing our data

In [17]:
# Using StandardScaler to normalize our data
scaler = StandardScaler()
scaler.fit(X_train_dash)

# transform
X_train_dash = pd.DataFrame(scaler.transform(X_train_dash),columns=X_train_dash.columns)
X_test_dash = pd.DataFrame(scaler.transform(X_test_dash),columns=X_test_dash.columns)

In [18]:
# Using StandardScaler to normalize our test data
test_data_dash = pd.DataFrame(scaler.transform(test_data_dash),columns=test_data_dash.columns)

# Training 

# Xgboost

In [19]:
# Using Xgboost
classifier = XGBClassifier(n_estimators=500,
                           max_depth=5,
                           learning_rate=0.15,
                           colsample_bytree=1,
                           subsample=1,
                           reg_alpha = 0.3,
                           gamma=10,
                           n_jobs=-1,
                           eval_metric='logloss',
                           use_label_encoder=False)

classifier.fit(X_train_dash, y_train)

y_train_pred = classifier.predict(X_train_dash)
y_train_prob = classifier.predict_proba(X_train_dash)[:,1]
y_val_pred = classifier.predict(X_test_dash)
y_val_prob = classifier.predict_proba(X_test_dash)[:,1]


# Calculating logloss score for our model
print(f'Train Logloss for the model -> {log_loss(y_train,y_train_prob)}')
print(f'Validation Logloss for the model -> {log_loss(y_val,y_val_prob)}')

print('-'*50)
# Calculating the AUC score for our model
print(f'Train AUC Score for the model -> {roc_auc_score(y_train, y_train_prob)}')
print(f'Validation AUC Score for the model -> {roc_auc_score(y_val, y_val_prob)}')

Train Logloss for the model -> 0.14337571574014177
Validation Logloss for the model -> 0.15469231195819308
--------------------------------------------------
Train AUC Score for the model -> 0.9890845850873322
Validation AUC Score for the model -> 0.9881158923367048


# Predictions on Test data

In [20]:
y_test_pred = classifier.predict(test_data_dash)
print(y_test_pred)

[0 1 1 1 1 0 1 1 1 1 0 0 0 0 0 0 0 0 1 0 1 1 1 0 1 0 0 1 0 0 1 0 0 0 0 0 0
 0 1 0 1 0 0 0 0 1 0 0 0 1 1 0 0 1 1 1 0 0 0 1 1 0 0 0 0 1 1 1 0 0 0 1 0 1
 1 1 0 0 0 0 0 1 1 1 0 0 0 0 1 0 0 0 0 0 0 1 1 0 1 1 0 0 0 1 1 0 0 0 0 1 0
 1 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 1 0 0 0 1 0 0 1 0 1 0 0 0 0 0 1 1 0
 1 1 0 1 1 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 0 0 0 1 0 1 0 0 1 0 1 1 0 0 0 1 0
 0 1 0 0 1 1 0 1 1 1 1 0 0 0 1 0 1 0 0 0 1 1 0 1 0 0 1 0 0 0 0 0 0 0 1 0 1
 0 0 0 1 1 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 1 0 0 1 1 1
 1 0 0 1 0 1 0 0 0 1 0 0 1 1 0 1 1 1 0 0 0 0 0 1 0 1 0 0 1 0 0 1 0 0 1 1 0
 0 0 0 1 0 1 0 0 0 0 0 1 0 1 1 1 0 0 1 0 0 0 0 0 0 0 0 1 0 1 0 1 0 0 0 1 1
 0 1 1 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 1 0 0 1 0 0 0 1 0 0 0 0 0 0 0 1 0 1 1
 1 1 0 0 1 0 0 1 1 1 1 1 0 0 0 0 1 0 0 1 1 1 0 1 1 1 0 0 0 1 1 1 1 1 1 1 0
 1 0 0 0 1 1 0 0 0 0 0 0 1 1 1 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 1 1 0 0 1 1 0
 0 0 0 1 1 0 1 1 1 0 0 1 0 0 1 1 0 0 1 1 0 1 0 0 1 1 0 0 1 0 0 1 1 1 1 1 0
 0 1 0 0 0 0 0 0 0 0 1 0 

In [21]:
y_test_prob = classifier.predict_proba(test_data_dash)[:,1]
print(y_test_prob)

[0.04236659 0.9799453  0.9817722  0.973751   0.978819   0.20681825
 0.9771714  0.7067784  0.9675239  0.983721   0.01044433 0.07185189
 0.21114746 0.01652415 0.05903362 0.11397144 0.02350036 0.16215698
 0.9698316  0.02088377 0.9759626  0.97873086 0.9744621  0.01000602
 0.77090746 0.06609312 0.1032019  0.84450877 0.06609312 0.01105931
 0.89442337 0.05260878 0.01652415 0.05201242 0.01225299 0.03658777
 0.4534763  0.02922887 0.5368635  0.46999773 0.9838521  0.09450296
 0.16215698 0.12399059 0.04577107 0.9401931  0.07101598 0.01805492
 0.03803167 0.9736239  0.85039765 0.10282127 0.01045498 0.9307046
 0.9787997  0.97803295 0.01926898 0.01777293 0.45117712 0.95659757
 0.97755414 0.163437   0.3375191  0.2114131  0.35615364 0.965411
 0.9304606  0.7690301  0.11353943 0.05201242 0.02389444 0.8761709
 0.05201242 0.5587642  0.9270567  0.97100306 0.0962676  0.01069072
 0.31966805 0.05201242 0.08003447 0.94516116 0.944969   0.9700033
 0.05201242 0.01652415 0.09344687 0.05201242 0.96715164 0.0128664
 